In [1]:
from coulson.interface import process_coordinates, gen_coords_for_mol, process_rdkit_mol
from coulson.ppp import calculate_exchange, calculate_dsp, homo_lumo_overlap, PPPCalculator
from coulson.draw import draw_mol   
from coulson.data import HARTREE_TO_EV
from morfeus.io import read_xyz

In [2]:
from coulson.parameters import MODERN, BEVERIDGE_HINZE

In [3]:
from rdkit import Chem

In [4]:
smiles = "C1=CC2=CC=CC2=C1"
#smiles = "c1cc2cccc3cccc(c1)-n-2-3"
mol = Chem.MolFromSmiles(smiles)
gen_coords_for_mol(mol)
input_data, mask = process_rdkit_mol(mol)

In [5]:
symbols, coordinates = read_xyz("../../data/azaphenalenes_xyz/1.xyz")
input_data, mask = process_coordinates(symbols, coordinates)

In [6]:
ppp = PPPCalculator(input_data, parametrization=BEVERIDGE_HINZE)

In [7]:
ppp.scf()

In [8]:
# Calcualte exchange integral
exchange = calculate_exchange(ppp)

# Calculate DSP contribution
dsp = calculate_dsp(ppp)

# Calculate HOMO-LUMO overlap
overlap = homo_lumo_overlap(ppp)

# Calculate S1 energies and oscillator strengths
ppp.ci(n_states=3)
energy_s1_cis = (ppp.ci_energies[1] - ppp.ci_energies[0])
    
oscillator_strength = ppp.oscillator_strengths[0]

# Calculate T1 energies
ppp.ci(n_states=3, multiplicity="triplet")
energy_t1_cis = (ppp.ci_energies[1] - ppp.ci_energies[0])

In [9]:
import itertools
import numpy as np

def calculate_dsp_old(ppp: PPPCalculator) -> float:
    """Calculate dynamic spin polarization difference with perturbation theory.

    Spin polarization difference between singlet and triplet HOMO->LUMO excited states.
    Approach from 10.1007/BF00549021. Negative values indicate singlet is more
    stabilized than triplet.

    Args:
        ppp: PPPCalculator object

    Returns:
        dsp: Stabilization of singlet over triplet (a.u.)
    """
    # Set up variables
    n_occupied = ppp.n_occupied
    n_virtual = ppp.n_virtual
    homo_idx = ppp.homo_idx
    lumo_idx = ppp.lumo_idx

    # Calculate MO integrals and Fock matrix in MO basis
    ppp._setup_mo_integrals()
    ppp._setup_fock_matrix_mo()

    # Generate all single excitations
    single_excitations = list(
        itertools.product(
            range(n_occupied - 1), range(n_occupied + 1, n_occupied + n_virtual)
        )
    )

    # Do perturbation
    s_1_all = []
    t_1_all = []
    t_2_all = []
    for i, j in single_excitations:
        k_x = ppp.mo_integrals[i, homo_idx, homo_idx, j]
        k_y = ppp.mo_integrals[i, lumo_idx, lumo_idx, j]
        gap = ppp.fock_matrix_mo[j, j] - ppp.fock_matrix_mo[i, i]
        s_1 = (3 / 2) * (k_x - k_y) ** 2 / gap
        t_1 = (1 / 2) * (k_x - k_y) ** 2 / gap
        t_2 = (k_x + k_y) ** 2 / gap
        s_1_all.append(s_1)
        t_1_all.append(t_1)
        t_2_all.append(t_2)

    # Sum contributions and calculate DSP
    s_1 = np.sum(s_1_all)
    t_1 = np.sum(t_1_all)
    t_2 = np.sum(t_2_all)
    dsp: float = -(s_1 - t_1 - t_2)

    return dsp

In [11]:
dsp, d = calculate_dsp(ppp, ci=True, energy_s_1=energy_s1_cis, energy_t_1=energy_t1_cis)
print(dsp)
dsp, d = calculate_dsp(ppp, ci=False)
print(dsp)
dsp_old = calculate_dsp_old(ppp)
print(dsp_old)

-0.017745669919750136
-0.01807595158617816
-0.01807595158617817


In [78]:
from IPython.display import SVG
from coulson.draw import draw_png_grid
from io import BytesIO

In [79]:
delta_st = energy_s1_cis - energy_t1_cis


print(f"ΔE_ST (CIS): {(delta_st + dsp) * 627.51:.3f} kcal/mol")
print(f"ΔE_ST (SCF): {(2 * exchange + dsp) * 627.51:.3f} kcal/mol")

for (i, j), dsp_ij in itertools.islice(d.items(), None):
    value = dsp_ij["dsp"] * 627.51
    if abs(value) > 0.01:
        print(f"{i} -> {j}: {value:.3f} kcal/mol")
    #png_i = draw_mol(mol=mol, properties=ppp.hc.coefficients[i], img_format="png")
    #png_j = draw_mol(mol=mol, properties=ppp.hc.coefficients[j], img_format="png")
    #fig, ax = draw_png_grid([BytesIO(i) for i in (png_i, png_j)], labels=[i, j])
    #display(fig)


ΔE_ST (CIS): -10.910 kcal/mol
ΔE_ST (SCF): -10.705 kcal/mol
0 -> 12: 0.078 kcal/mol
1 -> 8: -0.014 kcal/mol
1 -> 9: -0.026 kcal/mol
1 -> 10: -0.514 kcal/mol
1 -> 11: -0.279 kcal/mol
2 -> 8: -0.026 kcal/mol
2 -> 9: -0.014 kcal/mol
2 -> 10: -0.279 kcal/mol
2 -> 11: -0.514 kcal/mol
3 -> 12: -0.937 kcal/mol
4 -> 8: -0.016 kcal/mol
4 -> 9: -4.204 kcal/mol
4 -> 10: -0.010 kcal/mol
4 -> 11: -0.177 kcal/mol
5 -> 8: -4.205 kcal/mol
5 -> 9: -0.016 kcal/mol
5 -> 10: -0.177 kcal/mol
5 -> 11: -0.010 kcal/mol


In [47]:
sum(d.values())

-0.014286490840696775

-7.046913930792993